I will try to download snp500 data here then will modify them at each assets as csv files to save as data folder

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from yahoofinancials import YahooFinancials

I make ready the downloading function

In [15]:
def download_csv_data(ticker, start_date, end_date, freq, path):
    
    yahoo_financials = YahooFinancials(ticker)

    df = yahoo_financials.get_historical_price_data(start_date, end_date, freq)
    df = pd.DataFrame(df[ticker]['prices']).drop(['date'], axis=1) \
            .rename(columns={'formatted_date':'date'}) \
            .loc[:, ['date','open','high','low','close','volume']] \
            .set_index('date')
    df.index = pd.to_datetime(df.index)
    #df['dividend'] = 0
    #df['split'] = 1

    # save data to csv for later ingestion
    df.to_csv(path, header=True, index=True)
    



I took the ticker list here

In [12]:
import wikipedia as wp

# Scrape the table from the S&P 500 Wikipedia page
html = wp.page("List of S&P 500 companies").html().encode("UTF-8")
df = pd.read_html(html)[0]

# Extract the ticker symbols from the table
tickers = df['Symbol'].tolist()

# Retrieve historical data for each ticker symbol
start_date = "2018-01-01"
end_date = "2023-06-30"

In [13]:
tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BG',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',

now I will try to download the files in a for loop

In [20]:
for ticker in tickers:
    try:
        filename = f"data/daily/{ticker}.csv"
        download_csv_data(ticker=ticker, start_date=start_date, end_date=end_date, freq='daily', path=filename)
    except Exception as e:
        print(f"Error occurred while downloading data for ticker {ticker}: {str(e)}")
        continue
    print(ticker, "completed")


MMM completed
AOS completed
ABT completed
ABBV completed
ACN completed
ATVI completed
ADM completed
ADBE completed
ADP completed
AAP completed
AES completed
AFL completed
A completed
APD completed
AKAM completed
ALK completed
ALB completed
ARE completed
ALGN completed
ALLE completed
LNT completed
ALL completed
GOOGL completed
GOOG completed
MO completed
AMZN completed
AMCR completed
AMD completed
AEE completed
AAL completed
AEP completed
AXP completed
AIG completed
AMT completed
AWK completed
AMP completed
ABC completed
AME completed
AMGN completed
APH completed
ADI completed
ANSS completed
AON completed
APA completed
AAPL completed
AMAT completed
APTV completed
ACGL completed
ANET completed
AJG completed
AIZ completed
T completed
ATO completed
ADSK completed
AZO completed
AVB completed
AVY completed
AXON completed
BKR completed
BALL completed
BAC completed
BBWI completed
BAX completed
BDX completed
WRB completed
Error occurred while downloading data for ticker BRK.B: 'prices'
BBY comp

In [18]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('A.csv')

# Find the row with the specified date
target_date = '2022-06-17'  # Replace with the date you're looking for
target_row = df[df['date'] == target_date].iloc[0]

# Create a copy of the target row
new_row = target_row.copy()

# Modify the date of the new row
new_date = pd.to_datetime(target_row['date']) + pd.DateOffset(days=3)
new_row['date'] = new_date.strftime('%Y-%m-%d')  # Format the date as desired

# Append the new row to the DataFrame
df = df.append(new_row, ignore_index=True)
# Sort the DataFrame by the date column
df = df.sort_values('date').reset_index(drop=True)

# Save the modified DataFrame back to CSV
df.to_csv('A.csv', index=False)


/tmp/ipykernel_122297/3675819327.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


there is two missing values while ingesting, that are 2023-06-19 and 2022-06-20, so, I manually writting the value for the ingestion

In [21]:
import os


# Folder path containing the CSV files
folder_path = 'data/daily'

# List to store filenames with errors
error_files = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        try:
            # Read the CSV file
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)

            # Find the row with the specified date
            target_date = '2022-06-17'  # Replace with the date you're looking for
            target_rows = df[df['date'] == target_date]

            if len(target_rows) > 0:
                # Take the first matching row
                target_row = target_rows.iloc[0]

                # Create a copy of the target row
                new_row = target_row.copy()

                # Modify the date of the new row
                new_date = pd.to_datetime(target_row['date']) + pd.DateOffset(days=3)
                new_row['date'] = new_date.strftime('%Y-%m-%d')  # Format the date as desired

                # Append the new row to the DataFrame
                df = df.append(new_row, ignore_index=True)

                # Sort the DataFrame by the date column
                df = df.sort_values('date').reset_index(drop=True)

                # Save the modified DataFrame back to CSV with the original filename
                modified_file_path = os.path.join(folder_path, filename)
                df.to_csv(modified_file_path, index=False)

                print(f"Processed file: {filename}")
            else:
                print(f"No matching rows found for date {target_date} in file: {filename}")

        except Exception as e:
            print(f"Error occurred while processing file: {filename}")
            error_files.append(filename)

print("Error files:", error_files)


/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

Processed file: DLR.csv
Processed file: TSCO.csv
Processed file: LLY.csv
Processed file: CTLT.csv
Processed file: MCK.csv
Processed file: AKAM.csv
Processed file: AMD.csv
Processed file: STLD.csv
Processed file: AON.csv
Processed file: ETR.csv
Processed file: LEN.csv
Processed file: BIIB.csv
Processed file: ADSK.csv
Processed file: RJF.csv


/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

Processed file: DGX.csv
Processed file: XOM.csv
Processed file: ALGN.csv
Processed file: SBAC.csv
Processed file: CTAS.csv
Processed file: ANSS.csv
Processed file: LDOS.csv
Processed file: CZR.csv
Processed file: CTRA.csv
Processed file: EMR.csv
Processed file: RF.csv
Processed file: DG.csv
Processed file: SHW.csv
Processed file: AZO.csv


/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

...

/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

Processed file: PEAK.csv
Processed file: TFC.csv
Processed file: NXPI.csv
Processed file: EIX.csv
Processed file: COP.csv
Processed file: AWK.csv
Processed file: HSY.csv
Processed file: WBD.csv
Processed file: PNR.csv


/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

Processed file: MAR.csv
Processed file: AFL.csv
Processed file: APTV.csv
Processed file: TMO.csv
Processed file: MGM.csv
Processed file: GNRC.csv
Processed file: AMCR.csv
Processed file: INVH.csv
Processed file: CPB.csv
Processed file: PLD.csv
Processed file: BMY.csv
Processed file: CHD.csv
Processed file: O.csv
Processed file: KEY.csv
Processed file: CLX.csv
Processed file: IRM.csv
Processed file: AMAT.csv
Processed file: CEG.csv
Processed file: REG.csv
Processed file: BSX.csv
Error files: []


/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/4146860370.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

In [23]:
import os


# Folder path containing the CSV files
folder_path = 'data/daily'

# List to store filenames with errors
error_files = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        try:
            # Read the CSV file
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)

            # Find the row with the specified date
            target_date = '2023-06-16'  # Replace with the date you're looking for
            target_rows = df[df['date'] == target_date]

            if len(target_rows) > 0:
                # Take the first matching row
                target_row = target_rows.iloc[0]

                # Create a copy of the target row
                new_row = target_row.copy()

                # Modify the date of the new row
                new_date = pd.to_datetime(target_row['date']) + pd.DateOffset(days=3)
                new_row['date'] = new_date.strftime('%Y-%m-%d')  # Format the date as desired

                # Append the new row to the DataFrame
                df = df.append(new_row, ignore_index=True)

                # Sort the DataFrame by the date column
                df = df.sort_values('date').reset_index(drop=True)

                # Save the modified DataFrame back to CSV with the original filename
                modified_file_path = os.path.join(folder_path, filename)
                df.to_csv(modified_file_path, index=False)

                print(f"Processed file: {filename}")
            else:
                print(f"No matching rows found for date {target_date} in file: {filename}")

        except Exception as e:
            print(f"Error occurred while processing file: {filename}")
            error_files.append(filename)

print("Error files:", error_files)


/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

Processed file: DLR.csv
Processed file: TSCO.csv
Processed file: LLY.csv
Processed file: CTLT.csv
Processed file: MCK.csv
Processed file: AKAM.csv
Processed file: AMD.csv
Processed file: STLD.csv


/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

Processed file: AON.csv
Processed file: ETR.csv
Processed file: LEN.csv
Processed file: BIIB.csv
Processed file: ADSK.csv
Processed file: RJF.csv
Processed file: DGX.csv
Processed file: XOM.csv
Processed file: ALGN.csv
Processed file: SBAC.csv


/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

...

/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

Processed file: AFL.csv
Processed file: APTV.csv
Processed file: TMO.csv
Processed file: MGM.csv
Processed file: GNRC.csv
Processed file: AMCR.csv
Processed file: INVH.csv
Processed file: CPB.csv
Processed file: PLD.csv


/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

Processed file: BMY.csv
Processed file: CHD.csv
Processed file: O.csv
Processed file: KEY.csv
Processed file: CLX.csv
Processed file: IRM.csv
Processed file: AMAT.csv
Processed file: CEG.csv
Processed file: REG.csv
Processed file: BSX.csv
Error files: []


/tmp/ipykernel_122297/2733598170.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


finally the ingestion done!